# Step 1: Import necessary packages

In [5]:
from decouple import config
import brawlstats as brawl
import pandas as pd

# Step 2: Connect to the API and get my playerdata

In [82]:
try:
    TOKEN = config('TOKEN')
    client = brawl.Client(TOKEN)
    player = client.get_player("2GGYGV8YQ")
    print("Connected successfully and returned player: ",player.name)
except:
    print("an error occured with connection")

Connected successfully and returned player:  Zuki


# Step 3: Put the playerdata into a usable data structure

In [91]:
df = pd.DataFrame(player.brawlers)#read the brawlers list from my player data
df.drop(columns=['id'], inplace=True)#drop the brawler id columns
df = df[['name','trophies','gadgets','power', 'highest_trophies',   'rank', 'star_powers', ]]#reorder columns
for index, row in df.iterrows():#loop through the dataframe
    dicList = row["gadgets"]#get the list from the column
    for i in range(len(dicList)):#loop through the list
        if(type(dicList[i])!= str):#if the list item is a dictionary and not a string
            item = dicList[i]['name']#get the name from the dictionary
            dicList[i] = item#replace the list with just the name
    df.at[index,'gadgets'] = dicList#update the dataframe with the new list
    powerList = row["star_powers"]#DO THE SAME FOR GADGETS
    for i in range(len(powerList)):
        if(type(powerList[i])!= str):
            item = powerList[i]['name']
            powerList[i] = item
    df.at[index,'star_powers'] = powerList
df.sort_values(by=['trophies'], ascending=False)#sort by trophies, highest-lowest


,name,trophies,gadgets,power,highest_trophies,rank,star_powers
10,POCO,571,[TUNING FORK],10,573,21,"[SCREECHING SOLO, DA CAPO!]"
30,SURGE,509,[POWER SURGE],8,509,20,[]
1,COLT,488,[SPEEDLOADER],8,488,19,[]
21,BIBI,414,[VITAMIN BOOSTER],8,414,17,[]
14,DARRYL,407,[RECOILING ROTATOR],7,407,17,[]
19,ROSA,399,[],7,404,17,[]
6,JESSIE,383,"[RECOIL SPRING, SPARK PLUG]",7,383,17,[]
9,EL PRIMO,379,[],7,379,16,[]
0,SHELLY,368,"[CLAY PIGEONS, FAST FORWARD]",7,372,16,[]
2,BULL,364,[T-BONE INJECTOR],7,364,16,[]


# Step 4: Use the playerdata to find some meaningful information

### Meaningful information no.1: try to find who has a lower trophy count than expected which indicates which brawlers will be easier to gain trophies on.

in order to find the brawlers with low trophies, I have to define what the expected trophy count should be:
since power 10 is the maximum power, and is only achieved with the existence of a star power, we can say that a hero with a gadget and is power 10 should be able to acheieve the maximum trophie count (550 trophies is the limit of trophies as extras are removed every few weeks for rewards). we can then arbritrarily say that a brawler with ((powerlevel-1)+(1 if the brawler has a gadget)) x 10% is the percentage of trophies that a brawler is expected to have out of 550

In [108]:
#edit the dataframe to contain only necessary information
maxTrophies = 550
estimateDF = df.copy()
estimateDF.drop(columns=['star_powers','rank','highest_trophies'], inplace=True)
for index, row in estimateDF.iterrows():#loop through the dataframe
    gagExist = 0
    gList = row["gadgets"]#get the list from the column
    if(len(gList)>0):
        gagExist = 1
    estimateDF.at[index,'gadgets'] = gagExist
#find the estimate and insert it into the dataframe
for index, row in estimateDF.iterrows():
    powerLevel = row['power']
    gagExist = row["gadgets"]
    troph = row['trophies']
    estimate = ((powerLevel-1)+gagExist)/10
    trophyEstimate = estimate * maxTrophies
    estimateDF.at[index,'trophyEstimate'] = trophyEstimate
    estimateDF.at[index,'trophyError'] = troph-trophyEstimate
estimateDF.sort_values(by=['trophyError'])#sort by error, most-least
    

,name,trophies,gadgets,power,trophyEstimate,trophyError
31,COLETTE,0,0,7,330.0,-330.0
28,GALE,85,0,7,330.0,-245.0
11,BO,228,1,7,385.0,-157.0
25,MR. P,236,1,7,385.0,-149.0
4,RICO,216,0,7,330.0,-114.0
22,8-BIT,114,0,5,220.0,-106.0
13,PAM,183,0,6,275.0,-92.0
12,PIPER,132,0,5,220.0,-88.0
5,BARLEY,301,1,7,385.0,-84.0
17,GENE,146,0,5,220.0,-74.0
